In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Cargar los datos
data_url = 'https://github.com/watorres/ProyectoDeepLearning/raw/main/device_gnss.csv.zip'
df = pd.read_csv(data_url, compression='zip')

# Seleccionar las características relevantes
features = ['RawPseudorangeMeters', 'RawPseudorangeUncertaintyMeters', 'SvPositionXEcefMeters',
            'SvPositionYEcefMeters', 'SvPositionZEcefMeters', 'SvClockBiasMeters', 'SvClockDriftMetersPerSecond']
target = ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']

df = df[features + target].dropna()

# Dividir los datos en conjunto de entrenamiento y prueba
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True)

# Normalizar los datos
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_df.values)
test_data = scaler.transform(test_df.values)

# Preparar los datos de entrada y salida
train_X, train_y = train_data[:, :-3], train_data[:, -3:]
test_X, test_y = test_data[:, :-3], test_data[:, -3:]

# Reshape para LSTM (n_samples, timesteps, n_features)
timesteps = 1
train_X = train_X.reshape((train_X.shape[0], timesteps, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], timesteps, test_X.shape[1]))

# Definir el modelo LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(3))  # 3 para las coordenadas X, Y, Z
model.compile(loss='mse', optimizer='adam')

# Entrenar el modelo
model.fit(train_X, train_y, epochs=10, batch_size=32, validation_data=(test_X, test_y))

# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(test_X, test_y)
print('Loss en el conjunto de prueba:', loss)

# Hacer predicciones en el conjunto de prueba
predictions = model.predict(test_X)

# Desnormalizar las predicciones y los valores reales
predictions = scaler.inverse_transform(np.concatenate((test_X.reshape((test_X.shape[0], -1)), predictions), axis=1))[:, -3:]
actual = scaler.inverse_transform(np.concatenate((test_X.reshape((test_X.shape[0], -1)), test_y), axis=1))[:, -3:]

# Calcular los errores de distancia
errors = np.linalg.norm(predictions - actual, axis=1)
percentile_50 = np.percentile(errors, 50)
percentile_95 = np.percentile(errors, 95)

print('Error de percentil 50:', percentile_50)
print('Error de percentil 95:', percentile_95)

Epoch 1/10
1341/1341 [==============================] - 9s 4ms/step - loss: 0.1046 - val_loss: 0.0984
Epoch 2/10
1341/1341 [==============================] - 4s 3ms/step - loss: 0.0981 - val_loss: 0.0981
Epoch 3/10
1341/1341 [==============================] - 4s 3ms/step - loss: 0.0973 - val_loss: 0.0987
Epoch 4/10
1341/1341 [==============================] - 5s 4ms/step - loss: 0.0961 - val_loss: 0.0958
Epoch 5/10
1341/1341 [==============================] - 4s 3ms/step - loss: 0.0949 - val_loss: 0.0961
Epoch 6/10
1341/1341 [==============================] - 5s 4ms/step - loss: 0.0938 - val_loss: 0.0945
Epoch 7/10
1341/1341 [==============================] - 4s 3ms/step - loss: 0.0927 - val_loss: 0.0928
Epoch 8/10
1341/1341 [==============================] - 4s 3ms/step - loss: 0.0915 - val_loss: 0.0911
Epoch 9/10
1341/1341 [==============================] - 6s 4ms/step - loss: 0.0901 - val_loss: 0.0915
Epoch 10/10
336/336 [==============================] - 1s 2ms/step - loss: 0.0902


El resultado muestra el progreso del entrenamiento del modelo a lo largo de 10 épocas. Cada época representa un ciclo completo de alimentación hacia adelante y hacia atrás de los datos de entrenamiento a través de la red neuronal.

Para cada época, se muestra la pérdida (loss) tanto en el conjunto de entrenamiento como en el conjunto de validación (val_loss). La pérdida es una medida de la discrepancia entre las predicciones del modelo y los valores reales. El objetivo del entrenamiento es minimizar esta pérdida.

En este caso, podemos observar que la pérdida disminuye progresivamente a medida que avanzan las épocas tanto en el conjunto de entrenamiento como en el conjunto de validación. Esto indica que el modelo está aprendiendo a hacer predicciones más precisas a medida que se entrena.

Después de las 10 épocas, se evalúa el modelo en el conjunto de prueba y se muestra la pérdida en el conjunto de prueba (Loss en el conjunto de prueba). En este caso, la pérdida en el conjunto de prueba es 0.0902, lo que indica que el modelo tiene un buen desempeño en el conjunto de prueba.

Además, se calculan los errores de distancia utilizando las predicciones del modelo y los valores reales en el conjunto de prueba. El error de percentil 50 indica que el 50% de los errores de distancia son menores o iguales a 8421.9861 metros, mientras que el error de percentil 95 indica que el 95% de los errores de distancia son menores o iguales a 18874.2045 metros. Estos valores pueden ser utilizados para evaluar la precisión del modelo en la determinación de la ubicación de los teléfonos móviles.

En resumen, el modelo ha sido entrenado con éxito y muestra un buen desempeño en el conjunto de prueba, con una pérdida baja y errores de distancia aceptables. Sin embargo, es importante tener en cuenta el contexto del problema y las métricas de desempeño específicas para evaluar si estos resultados son satisfactorios para el objetivo del proyecto